# Salary Predictions Based on Job Descriptions

# Part 1 - DEFINE

### <font color=Firebrick>---- 1 Define the problem ----</font>

<font color=blue>**This Jupyter Notebook loads, explores, and visualizes the salary prediction datasets. Also, builds and tests several predictive models, and then makes salary predictions on test data using the best model.**</font>

In [48]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn as sk
from sklearn.utils import shuffle

%matplotlib inline
#Persoal info 
__author__ = "Bhargavsinh Ravalji"

## Part 2 - DISCOVER

### <font color=Firebrick>---- 2 Load the data and Examine the data----</font>

In [49]:
#Load csv file to pd dataframe
def load_data(file):
    return pd.read_csv(file)
train_feature_df = load_data('train_features.csv')
train_target_df = load_data('train_salaries.csv')
test_feature_df = load_data('test_features.csv')

#### <font color=green>*Examine the data*</font>

In [50]:
train_feature_df.head(10)

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
0,JOB1362684407687,COMP37,CFO,MASTERS,MATH,HEALTH,10,83
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73
2,JOB1362684407689,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38
3,JOB1362684407690,COMP38,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17
4,JOB1362684407691,COMP7,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16
5,JOB1362684407692,COMP15,MANAGER,DOCTORAL,COMPSCI,FINANCE,2,31
6,JOB1362684407693,COMP15,CFO,NONE,NONE,HEALTH,23,24
7,JOB1362684407694,COMP24,JUNIOR,BACHELORS,CHEMISTRY,EDUCATION,9,70
8,JOB1362684407695,COMP20,JANITOR,HIGH_SCHOOL,NONE,EDUCATION,1,54
9,JOB1362684407696,COMP41,VICE_PRESIDENT,BACHELORS,CHEMISTRY,AUTO,17,68


In [51]:
train_target_df.head(10)

,jobId,salary
0,JOB1362684407687,130
1,JOB1362684407688,101
2,JOB1362684407689,137
3,JOB1362684407690,142
4,JOB1362684407691,163
5,JOB1362684407692,113
6,JOB1362684407693,178
7,JOB1362684407694,73
8,JOB1362684407695,31
9,JOB1362684407696,104


In [52]:
test_feature_df.head(10)

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
0,JOB1362685407687,COMP33,MANAGER,HIGH_SCHOOL,NONE,HEALTH,22,73
1,JOB1362685407688,COMP13,JUNIOR,NONE,NONE,AUTO,20,47
2,JOB1362685407689,COMP10,CTO,MASTERS,BIOLOGY,HEALTH,17,9
3,JOB1362685407690,COMP21,MANAGER,HIGH_SCHOOL,NONE,OIL,14,96
4,JOB1362685407691,COMP36,JUNIOR,DOCTORAL,BIOLOGY,OIL,10,44
5,JOB1362685407692,COMP40,CTO,MASTERS,COMPSCI,FINANCE,6,23
6,JOB1362685407693,COMP32,SENIOR,MASTERS,COMPSCI,SERVICE,6,32
7,JOB1362685407694,COMP11,CEO,BACHELORS,BIOLOGY,SERVICE,9,73
8,JOB1362685407695,COMP39,MANAGER,DOCTORAL,PHYSICS,HEALTH,6,78
9,JOB1362685407696,COMP22,VICE_PRESIDENT,NONE,NONE,AUTO,23,97


In [53]:
train_feature_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
jobId                  1000000 non-null object
companyId              1000000 non-null object
jobType                1000000 non-null object
degree                 1000000 non-null object
major                  1000000 non-null object
industry               1000000 non-null object
yearsExperience        1000000 non-null int64
milesFromMetropolis    1000000 non-null int64
dtypes: int64(2), object(6)
memory usage: 61.0+ MB


In [54]:
train_feature_df.describe()

,yearsExperience,milesFromMetropolis
count,1000000.000000,1000000.000000
mean,11.992386,49.529260
std,7.212391,28.877733
min,0.000000,0.000000
25%,6.000000,25.000000
50%,12.000000,50.000000
75%,18.000000,75.000000
max,24.000000,99.000000


#### <font color=green>*Merge two dataframes*</font>

In [55]:
#Consolidate training data: Performing inner join and getting uniq rows from both dataframe
def consolidate_data(df1, df2, key=None, left_index=False, right_index=False):
    return pd.merge(left=df1, right=df2, how='inner', on=key, left_index=left_index, right_index=right_index) 

In [56]:
raw_train_df = consolidate_data(train_feature_df, train_target_df, key='jobId') 
raw_train_df.head(5)

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary
0,JOB1362684407687,COMP37,CFO,MASTERS,MATH,HEALTH,10,83,130
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73,101
2,JOB1362684407689,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38,137
3,JOB1362684407690,COMP38,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17,142
4,JOB1362684407691,COMP7,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16,163


### <font color=Firebrick>---- 3 Clean the data ----</font>

#### <font color=green>*Check for Duplicates*</font>

In [57]:
train_feature_df.duplicated().sum()

0

In [58]:
train_target_df.duplicated().sum()

0

In [61]:
test_feature_df.duplicated().sum()

0

#### <font color=green>*Check invalid or corrupt data and remove it*</font>

In [62]:
'''remove rows that contain salary <= 0 or duplicate job IDs'''
def clean_data(raw_df):
    clean_df = raw_df.drop_duplicates(subset='jobId')
    clean_df = clean_df[clean_df.salary > 0]
    return clean_df

In [63]:
'''Shuffle, and reindex training data -- shuffling improves cross-validation accuracy'''
clean_train_df = shuffle(clean_data(raw_train_df)).reset_index()
clean_train_df.head(5)

,index,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary
0,75978,JOB1362684483665,COMP48,VICE_PRESIDENT,HIGH_SCHOOL,NONE,OIL,7,48,112
1,899895,JOB1362685307582,COMP0,CFO,DOCTORAL,BIOLOGY,OIL,6,35,123
2,561622,JOB1362684969309,COMP32,CEO,DOCTORAL,PHYSICS,HEALTH,10,53,181
3,90891,JOB1362684498578,COMP2,JUNIOR,DOCTORAL,ENGINEERING,FINANCE,20,57,178
4,787257,JOB1362685194944,COMP62,MANAGER,NONE,NONE,EDUCATION,13,76,63


### <font color=Firebrick>---- 4 Explore the data (EDA) ----</font>

In [3]:
#summarize each feature variable
#summarize the target variable
#look for correlation between each feature and the target
#look for correlation between features

### <font color=Firebrick>---- 5 Establish a baseline ----</font>

In [5]:
#select a reasonable metric (MSE in this case)
#create an extremely simple model and measure its efficacy
#e.g. use "average salary" for each industry as your model and then measure MSE
#during 5-fold cross-validation

### <font color=Firebrick>---- 6 Hypothesize solution ----</font>

In [ ]:
#brainstorm 3 models that you think may improve results over the baseline model based
#on your 

Brainstorm 3 models that you think may improve results over the baseline model based on your EDA and explain why they're reasonable solutions here.

Also write down any new features that you think you should try adding to the model based on your EDA, e.g. interaction variables, summary statistics for each group, etc

## Part 3 - DEVELOP

You will cycle through creating features, tuning models, and training/validing models (steps 7-9) until you've reached your efficacy goal

#### Your metric will be MSE and your goal is:
 - <360 for entry-level data science roles
 - <320 for senior data science roles

### <font color=Firebrick>---- 7 Engineer features  ----</font>

In [37]:
#make sure that data is ready for modeling
#create any new features needed to potentially enhance model

### <font color=Firebrick>---- 8 Create models ----</font>

In [38]:
#create and tune the models that you brainstormed during part 2

### <font color=Firebrick>---- 9 Test models ----</font>

In [39]:
#do 5-fold cross validation on models and measure MSE

### <font color=Firebrick>---- 10 Select best model  ----</font>

In [40]:
#select the model with the lowest error as your "prodcuction" model

## Part 4 - DEPLOY

### <font color=Firebrick>---- 11 Automate pipeline ----</font>

In [ ]:
#write script that trains model on entire training set, saves model to disk,
#and scores the "test" dataset

### <font color=Firebrick>---- 12 Deploy solution ----</font>

In [16]:
#save your prediction to a csv file or optionally save them as a table in a SQL database
#additionally, you want to save a visualization and summary of your prediction and feature importances
#these visualizations and summaries will be extremely useful to business stakeholders

### <font color=Firebrick>---- 13 Measure efficacy ----</font>

We'll skip this step since we don't have the outcomes for the test data